**Vamos a hacer que cada línea del dataset sea un párrafo de la web scrapeada. Vamos a obtener una nueva lista de palabras confusas y entrenar el modelo luego de haberlas excluido**

In [1]:
clases = 5

In [2]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pd
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pickle
def save_obj(obj, name):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
       with open(name + '.pkl', 'rb') as f:
           return pickle.load(f)

In [4]:
 def limpiarFecha(st):    
        import re
        p1 = r"((\d+)\s(bc|ac)\s)"
        p2 = r"((\d+)\s(bc|ac))$"
        patron = re.compile("|".join([p1,p2]),re.IGNORECASE)
        for i in range(len(st)):
            #print("ciclo",i)
            #print(st)
            if re.search(p1 , st[i:] , re.IGNORECASE):
                var = patron.search(st[i:] , re.IGNORECASE)
                # print(var,len(var[1].strip()),var[1],"|",var[2],"|",var[3])
                new = "".join([var[2],var[3]])
                st  = st.replace(var[1].strip(),new,1)
                print(st)
        return st

In [193]:
def limpiarTextoCrudo(st):
    res = []
    for t in range(500):
        st = st.replace("[" + str(t) + "]", " ")
    for letra in ["a","b","d","e","f","g","h","i","j"]:
        st = st.replace("[" + letra + "]", " ")
    caracteresARemover = ["(",")","[","]",".",",",'"',"“"]
    for signo in caracteresARemover:
        st = st.replace(signo," ")
    st = st.split()
    for i in range(len(st)):
        if st[i].isdecimal() and i < len(st) -1 and (
                                   st[i+1].lower() == "ad" or 
                                   st[i+1].lower() == "bc" or 
                                   st[i+1].lower() == "ac" or 
                                   st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
        elif st[i].lower() not in {"ad","bc","ac","dc"}:
            res.append(st[i])
    if st[-1] not in res  and st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

In [81]:
limpiarEdad("The Roman–Parthian Wars (54BC – 217 AD) were a series of conflicts  [1]   200 ad. 300 bc)")

'The Roman–Parthian Wars 54BC – 217AD were a series of conflicts 200ad 300bc'

In [54]:
limpiarEdad("a in 42 BC. campaign 42 bc) against Parthia, but his assassination averted the war. During the ensuing Liberators' civil war, the Parthians actively supported Brutus and Cassius, sending a contingent which fought with them at the Battle of Philippi in 42 BC.[11]\n")

"a in 42BC. BC. campaign 42bc) bc) against Parthia, but his assassination averted the war. During ensuing Liberators' civil war, Parthians actively supported Brutus and Cassius, sending contingent which fought with them at Battle of Philippi 42 BC.[11]"

In [51]:
class Textos:
    
    def __init__(self, clases, webs):
        self.clases = clases
        self.lista_dataset = []
        self.crear_dataframe(clases, webs)

    def crear_dataframe(self, clases, webs):
        self.lista_dataset.append('TextoCrudo')
        self.lista_dataset.append('TextoLimpio')

        for i in range(1, clases + 1):
            self.lista_dataset.append('Clase' + str(i))

        self.lista_dataset.append('Predicción')
        self.lista_dataset.append('CLASE')
        self.dataframe = pd.DataFrame(columns=self.lista_dataset)
        self.webs = pd.ExcelFile(webs)
        df = self.webs.parse(self.webs.sheet_names[0])
        self.__clases_df = set(df["CÓDIGO"].value_counts().index)

        for i in df.index:
            print(f"Creando Dataframe {i+1} de {len(df.index)}")
            url = df["URL"][i]
            req = get(url)
            statusCode = req.status_code
            html = BeautifulSoup(req.text, "html.parser")

            if statusCode == 200:
                paragraphs = html.select("p")
                cont = 0

                for para in paragraphs:
                    if len(para.text) < 15:
                        continue
                    cont += 1
                    self.dataframe = self.dataframe.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)

                clear_output(wait=True)

    def limpiar(textoCrudo):
        for i in range(100):
            textoCrudo = textoCrudo.replace("[" + str(i) + "]", "")
        
        for i in ["a","b","c"]:
            textoCrudo = textoCrudo.replace("[" + i + "]", "")
        
        stopwords_english = stopwords.words('english')
        stop        = stopwords.words('english')
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        stemmer = PorterStemmer()    
        texto_tokens = tokenizer.tokenize(textoCrudo)
        sufijos_fecha = ["bc","ac"]

        for i in range(len(texto_tokens.copy())):
            if i > len(texto_tokens) - 1:
                break
            if i !=0:
                if texto_tokens[i] in sufijos_fecha and texto_tokens[i - 1].isnumeric():
                    texto_tokens[i] = texto_tokens[i - 1] + texto_tokens[i]     
                    texto_tokens.pop(i - 1)
                           
        texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation]  
        texto_stem   = [stemmer.stem(word) for word in texto_clean] 
        return texto_stem       
      

In [9]:
lista_dataset = []
lista_dataset.append('TextoCrudo')
lista_dataset.append('TextoLimpio')
for i in range(1, clases + 1):
    lista_dataset.append('Clase' + str(i))
lista_dataset.append('Predicción')
lista_dataset.append('CLASE')
dataFrame = pd.DataFrame(columns = lista_dataset)

In [10]:
textos = dataFrame

In [83]:
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars (54 – 217AD) AD) were ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and Roman ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"In 113AD, AD, the Roman Emperor Trajan made ea...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"In 195, another Roman invasion of Mesopotamia ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,"In February 2019, scientists reported evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [ ]:
webs = pd.ExcelFile("Webs.xls")

In [ ]:
df = webs.parse(webs.sheet_names[0])
clases_df = set(df["CÓDIGO"].value_counts().index)
clases_df

In [ ]:
df = df.iloc[12:60,:]

In [ ]:
df = df.drop([55,56,57])
df

In [37]:
df.to_csv("miniWEBs.csv")

In [38]:
df = pd.read_csv("miniWEBs.csv")
df = df.iloc[:,1:]
df

,URL,NOMBRE ARCHIVO,CLASE,CÓDIGO
0,https://en.wikipedia.org/wiki/Roman%E2%80%93Pa...,Roman%E2%80%93Parthian_Wars,EDAD ANTIGUA,1
1,https://en.wikipedia.org/wiki/Celts,Celts,EDAD ANTIGUA,1
2,https://en.wikipedia.org/wiki/Early_Middle_Ages,Early_Middle_Ages,EDAD MEDIA,2
3,https://en.wikipedia.org/wiki/Late_Middle_Ages,/Late_Middle_Ages,EDAD MEDIA,2
4,https://en.wikipedia.org/wiki/Philip_II_of_Spain,Philip_II_of_Spain,EDAD MODERNA,3
5,https://en.wikipedia.org/wiki/Louis_XIV,Louis_XIV,EDAD MODERNA,3
6,https://en.wikipedia.org/wiki/Spanish_American...,Spanish_American_wars_of_independence,EDAD CONTEMPORÁNEA,4
7,https://en.wikipedia.org/wiki/World_War_I,World_War_I,EDAD CONTEMPORÁNEA,4
8,https://en.wikipedia.org/wiki/Prehistory,Prehistory,PREHISTORIA,5
9,https://en.wikipedia.org/wiki/Paleolithic,Paleolithic,PREHISTORIA,5


In [13]:
for i in df.index:
    print(f"Creando Dataframe {i+1} de {len(df.index)}")
    url = df["URL"][i]
    req = get(url)
    statusCode = req.status_code
    if statusCode == 200:
        html = BeautifulSoup(req.text, "html.parser")
        paragraphs = html.select("p")
        cont = 0
        for para in paragraphs:
            if len(para.text) < 40:
                continue
            cont += 1
            textos = textos.append({'TextoCrudo': para.text, 'CLASE': df["CÓDIGO"][i]}, ignore_index=True)
    clear_output(wait=True)
textos 

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars (54 BC – 217 AD) were ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,"In 113 AD, the Roman Emperor Trajan made easte...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,"In 195, another Roman invasion of Mesopotamia ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,"In February 2019, scientists reported evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [14]:
textos.sample(10) 

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
458,"By the time of the Treaty of Ryswick, the Span...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
876,The Paleolithic is often held to finish at the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
810,"In 1915 John McCrae, a Canadian army doctor, w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
904,Upper Paleolithic humans produced works of art...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
662,Russian armies generally had success in the Ca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
681,The Czechoslovak Legion fought on the side of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
803,"Like all the armies of mainland Europe, Austri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
905,R. Dale Guthrie[77] has studied not only the m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
431,France also attempted to participate actively ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
653,"Faced with Russia in the east, Austria-Hungary...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [249]:
textos.to_csv("MiniHistoriaTextosCrudos.csv")

In [250]:
textos = pd.read_csv("MiniHistoriaTextosCrudos.csv")
textos = textos.iloc[:,1:]
textos.head(10)

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,By the same time the Parthians started their r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,After 90BC the Parthian power was diminished b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,When Pompey took charge of the war in the East...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,In 53BC Crassus led an invasion of Mesopotamia...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,The following year the Parthians launched raid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [40]:
textos["TextoCrudo"][10]

"During Caesar's civil war the Parthians made no move, but maintained relations with Pompey. After his defeat and death, a force under Pacorus came to the aid of the Pompeian general Caecilius Bassus, who was besieged at Apamea Valley by the Caesarian forces. With the civil war over, Julius Caesar elaborated plans for a campaign against Parthia, but his assassination averted the war. During the ensuing Liberators' civil war, the Parthians actively supported Brutus and Cassius, sending a contingent which fought with them at the Battle of Philippi in 42 BC.[11]\n"

In [251]:
for i in textos.index:
    clear_output(wait=True)        
    print(f"Limpiando Texto Crudo {i+1} de {len(textos.index)}")
    textos["TextoCrudo"][i] = limpiarTextoCrudo(textos["TextoCrudo"][i])
textos

Limpiando Texto Crudo 922 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [248]:
conta = 0
for i in range(1, len(textos.index)):
    conta += 1
    if conta > 4000:
        break
    print()
    print()
    print(textos["TextoCrudo"][i])
    print()
    print(limpiarTextoCrudo(textos["TextoCrudo"][i]))



Battles between the Parthian Empire and the Roman Republic began in 54BC This first incursion against Parthia was repulsed notably at the Battle of Carrhae 53BC During the Roman Liberators' civil war of the 1st Century the Parthians actively supported Brutus and Cassius invading Syria and gaining territories in the Levant However the conclusion of the second Roman civil war brought a revival of Roman strength in Western Asia

Battles between the Parthian Empire and the Roman Republic began in 54BC This first incursion against Parthia was repulsed notably at the Battle of Carrhae 53BC During the Roman Liberators' civil war of the 1st Century the Parthians actively supported Brutus and Cassius invading Syria and gaining territories in the Levant However the conclusion of the second Roman civil war brought a revival of Roman strength in Western Asia


In 113AD the Roman Emperor Trajan made eastern conquests and the defeat of Parthia a strategic priority and successfully overran the Part


The return to systemic agriculture coincided with the introduction of a new social system called feudalism This system featured a hierarchy of reciprocal obligations Each man was bound to serve his superior in return for the latter's protection This made for confusion of territorial sovereignty since allegiances were subject to change over time and were sometimes mutually contradictory Feudalism allowed the state to provide a degree of public safety despite the continued absence of bureaucracy and written records Even land ownership disputes were decided based solely on oral testimony Territoriality was reduced to a network of personal allegiances


The Viking Age spans the period roughly between the late 8th and mid-11th centuries in Scandinavia and Britain following the Germanic Iron Age and the Vendel Age in Sweden During this period the Vikings Scandinavian warriors and traders raided and explored most parts of Europe south-western Asia northern Africa and north-eastern North Amer

In 1558 Turkish admiral Piyale Pasha captured the Balearic Islands especially inflicting great damage on Menorca and enslaving many while raiding the coasts of the Spanish mainland Philip appealed to the Pope and other powers in Europe to bring an end to the rising Ottoman threat Since his father's losses against the Ottomans and against Hayreddin Barbarossa in 1541 the major European sea powers in the Mediterranean namely the Spanish Crown and Venice became hesitant in confronting the Ottomans The myth of Turkish invincibility was becoming a popular story causing fear and panic among the people


In 1560 Philip II organised a Holy League between the Spanish kingdoms and the Republic of Venice the Republic of Genoa the Papal States the Duchy of Savoy and the Knights of Malta The joint fleet was assembled at Messina and consisted of 200 ships 60 galleys and 140 other vessels carrying a total of 30 000 soldiers under the command of Giovanni Andrea Doria nephew of the famous Genoese admir


French military superiority might have allowed him to press for more advantageous terms Thus his generosity to Spain with regard to Catalonia has been read as a concession to foster pro-French sentiment and may ultimately have induced King Charles II to name Louis's grandson Philip Duke of Anjou heir to the Spanish throne In exchange for financial compensation France renounced its interests in the Electorate of Cologne and the Palatinate Lorraine which had been occupied by the French since 1670 was returned to its rightful Duke Leopold albeit with a right of way to the French military William and Mary were recognised as joint sovereigns of the British Isles and Louis withdrew support for James II The Dutch were given the right to garrison forts in the Spanish Netherlands that acted as a protective barrier against possible French aggression Though in some respects the Treaty of Ryswick may appear a diplomatic defeat for Louis since he failed to place client rulers in control of the Pal

The practical details of these alliances were limited since their primary purpose was to ensure cooperation between the three Imperial Powers and to isolate France Attempts by Britain in 1880 to resolve colonial tensions with Russia and diplomatic moves by France led to Bismarck reforming the League in 1881 When the League finally lapsed in 1887 it was replaced by the Reinsurance Treaty a secret agreement between Germany and Russia to remain neutral if either were attacked by France or Austria-Hungary


In 1890 the new German Emperor Kaiser Wilhelm II forced Bismarck to retire and was persuaded not to renew the Reinsurance Treaty by the new Chancellor Leo von Caprivi This allowed France to counteract the Triple Alliance with the Franco-Russian Alliance of 1894 and the 1904 Entente Cordiale with Britain while in 1907 Britain and Russia signed the Anglo-Russian Convention The agreements did not constitute formal alliances but by settling long-standing colonial disputes they made British 

About eight million men surrendered and were held in POW camps during the war All nations pledged to follow the Hague Conventions on fair treatment of prisoners of war and the survival rate for POWs was generally much higher than that of combatants at the front Individual surrenders were uncommon; large units usually surrendered en masse At the Siege of Maubeuge about 40 000 French soldiers surrendered at the battle of Galicia Russians took about 100 000 to 120 000 Austrian captives at the Brusilov Offensive about 325 000 to 417 000 Germans and Austrians surrendered to Russians and at the Battle of Tannenberg 92 000 Russians surrendered When the besieged garrison of Kaunas surrendered in 1915 some 20 000 Russians became prisoners at the battle near Przasnysz February–March 1915 14 000 Germans surrendered to Russians and at the First Battle of the Marne about 12 000 Germans surrendered to the Allies 25–31% of Russian losses as a proportion of those captured wounded or killed were to pri

In [196]:
textos.to_csv("MiniHistoriaTextosLimpios.csv")

In [308]:
textos = pd.read_csv("MiniHistoriaTextosLimpios.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [199]:
stopwords_english = stopwords.words('english')
stop        = stopwords.words('english')
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stemmer = PorterStemmer()    
texto_tokens = tokenizer.tokenize(textoCrudo)

In [256]:
cont = 0
for i in textos.index:
    cont += 1
    if cont > 1:
        break
    print()    
    # texto_tokens = tokenizer.tokenize(textos["TextoCrudo"][i])
    texto_tokens = textos["TextoCrudo"][i].lower().split()
    texto_clean  = [word for word in texto_tokens if word not in stop and word not in string.punctuation]
    texto_stem   = [stemmer.stem(word) for word in texto_clean]
    textos["TextoLimpio"][i] = texto_stem
    print()
    print()
    print(textos["TextoCrudo"][i])
    print(texto_tokens)
    print() 
    print (texto_clean)
    print() 
    print (texto_stem)
textos




The Roman–Parthian Wars 54BC – 217AD were a series of conflicts between the Parthian Empire and the Roman Republic and Roman Empire It was the first series of conflicts in what would be 682 years of Roman–Persian Wars
['the', 'roman–parthian', 'wars', '54bc', '–', '217ad', 'were', 'a', 'series', 'of', 'conflicts', 'between', 'the', 'parthian', 'empire', 'and', 'the', 'roman', 'republic', 'and', 'roman', 'empire', 'it', 'was', 'the', 'first', 'series', 'of', 'conflicts', 'in', 'what', 'would', 'be', '682', 'years', 'of', 'roman–persian', 'wars']

['roman–parthian', 'wars', '54bc', '–', '217ad', 'series', 'conflicts', 'parthian', 'empire', 'roman', 'republic', 'roman', 'empire', 'first', 'series', 'conflicts', 'would', '682', 'years', 'roman–persian', 'wars']

['roman–parthian', 'war', '54bc', '–', '217ad', 'seri', 'conflict', 'parthian', 'empir', 'roman', 'republ', 'roman', 'empir', 'first', 'seri', 'conflict', 'would', '682', 'year', 'roman–persian', 'war']


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"[roman–parthian, war, 54bc, –, 217ad, seri, co...",NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,"[battl, parthian, empir, roman, republ, began,...",NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"[In, 113ad, roman, emperor, trajan, made, east...",NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"[In, 195, anoth, roman, invas, mesopotamia, be...",NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"[after, triumph, seleucid–parthian, war, annex...",NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"[paleolith, peopl, suffer, less, famin, malnut...",NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,"[large-seed, legum, part, human, diet, long, n...",NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,"[In, februari, 2019, scientist, report, evid, ...",NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,"[anthropologist, tim, white, suggest, cannib, ...",NaN,NaN,NaN,NaN,NaN,NaN,5


In [257]:
for i in textos.index:
    # texto_tokens = tokenizer.tokenize(textos["TextoCrudo"][i])
    texto_tokens = textos["TextoCrudo"][i].lower().split()
    texto_clean  = [word for word in texto_tokens if word not in stop and word not in string.punctuation]
    texto_stem   = [stemmer.stem(word) for word in texto_clean]
    textos["TextoLimpio"][i] = texto_stem
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"[roman–parthian, war, 54bc, –, 217ad, seri, co...",NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,"[battl, parthian, empir, roman, republ, began,...",NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"[113ad, roman, emperor, trajan, made, eastern,...",NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"[195, anoth, roman, invas, mesopotamia, began,...",NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"[triumph, seleucid–parthian, war, annex, larg,...",NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"[paleolith, peopl, suffer, less, famin, malnut...",NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,"[large-seed, legum, part, human, diet, long, n...",NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,"[februari, 2019, scientist, report, evid, base...",NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,"[anthropologist, tim, white, suggest, cannib, ...",NaN,NaN,NaN,NaN,NaN,NaN,5


In [209]:
type(textos["TextoCrudo"][0])

str

In [210]:
textos["TextoLimpio"][0]

['the',
 'roman–parthian',
 'war',
 '54bc',
 '–',
 '217ad',
 'seri',
 'conflict',
 'parthian',
 'empir',
 'roman',
 'republ',
 'roman',
 'empir',
 'It',
 'first',
 'seri',
 'conflict',
 'would',
 '682',
 'year',
 'roman–persian',
 'war']

In [258]:
textos.to_csv("MiniHistoriaTextosProcesados.csv")

In [309]:
textos = pd.read_csv("MiniHistoriaTextosProcesados.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"['113ad', 'roman', 'emperor', 'trajan', 'made'...",NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"['195', 'anoth', 'roman', 'invas', 'mesopotami...",NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"['paleolith', 'peopl', 'suffer', 'less', 'fami...",NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,"['large-seed', 'legum', 'part', 'human', 'diet...",NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,"['februari', '2019', 'scientist', 'report', 'e...",NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,"['anthropologist', 'tim', 'white', 'suggest', ...",NaN,NaN,NaN,NaN,NaN,NaN,5


In [223]:
textos["TextoLimpio"][0]

"['the', 'roman–parthian', 'war', '54bc', '–', '217ad', 'seri', 'conflict', 'parthian', 'empir', 'roman', 'republ', 'roman', 'empir', 'It', 'first', 'seri', 'conflict', 'would', '682', 'year', 'roman–persian', 'war']"

In [221]:
type(textos.loc[:,"TextoLimpio"][0])

str

In [225]:
def limpiarWord(word):
    word = word[:-1]
    return word

In [260]:
frecuencias = defaultdict(int)
listaPalabras = []
for i in textos.index:
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        listaPalabras.append(word)    
        frecuencias[(word, textos["CLASE"][i])] = frecuencias.get((word, textos["CLASE"][i]), 0) + 1    
    clear_output( wait = True )
    print(f"Creando Frecuencias {i} de {len(textos.index)}")    
vocabulario = set(listaPalabras)    
frecuencias

Creando Frecuencias 921 de 922


defaultdict(int,
            {('roman–parthian', 1): 4,
             ('war', 1): 34,
             ('54bc', 1): 2,
             ('–', 1): 6,
             ('217ad', 1): 1,
             ('seri', 1): 4,
             ('conflict', 1): 5,
             ('parthian', 1): 48,
             ('empir', 1): 16,
             ('roman', 1): 129,
             ('republ', 1): 3,
             ('first', 1): 15,
             ('would', 1): 11,
             ('682', 1): 1,
             ('year', 1): 9,
             ('roman–persian', 1): 1,
             ('battl', 1): 15,
             ('began', 1): 10,
             ('incurs', 1): 1,
             ('parthia', 1): 14,
             ('repuls', 1): 1,
             ('notabl', 1): 2,
             ('carrha', 1): 3,
             ('53bc', 1): 2,
             ('liberators', 1): 2,
             ('civil', 1): 6,
             ('1st', 1): 9,
             ('centuri', 1): 53,
             ('activ', 1): 3,
             ('support', 1): 5,
             ('brutu', 1): 3,
             ('ca

In [261]:
len(frecuencias)

14994

In [262]:
len(vocabulario)

9070

In [263]:
"the" in vocabulario

False

In [264]:
save_obj(frecuencias, "Frecuencias")
save_obj(vocabulario, "Vocabulario")

In [265]:
frecuencias = load_obj("Frecuencias")
vocabulario = load_obj("Vocabulario")

In [266]:
def etiquetas(palabras,frecuencias,clases):
    clases = 5 
    import numpy as np
    x = np.zeros((1, clases + 1))
    for palabra in palabras:
        for t in range(1, clases + 1):
            x[0,t] += frecuencias.get((palabra, t),0)
    return x

In [267]:
for i in textos.index:
    print(i)
    palabras = []    
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 921 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",477.0,601.0,450.0,1131.0,89.0,NaN,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",980.0,926.0,571.0,1336.0,117.0,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"['113ad', 'roman', 'emperor', 'trajan', 'made'...",1079.0,733.0,508.0,787.0,68.0,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"['195', 'anoth', 'roman', 'invas', 'mesopotami...",448.0,435.0,161.0,277.0,31.0,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",483.0,594.0,523.0,933.0,196.0,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"['paleolith', 'peopl', 'suffer', 'less', 'fami...",322.0,476.0,436.0,728.0,1978.0,NaN,5
918,Large-seeded legumes were part of the human di...,"['large-seed', 'legum', 'part', 'human', 'diet...",786.0,830.0,842.0,1353.0,4099.0,NaN,5
919,In February 2019 scientists reported evidence ...,"['februari', '2019', 'scientist', 'report', 'e...",372.0,785.0,551.0,1174.0,3593.0,NaN,5
920,Anthropologists such as Tim White suggest that...,"['anthropologist', 'tim', 'white', 'suggest', ...",232.0,336.0,316.0,481.0,2008.0,NaN,5


In [241]:
textos["TextoCrudo"][0]

'The Roman–Parthian Wars 54BC – 217AD were a series of conflicts between the Parthian Empire and the Roman Republic and Roman Empire It was the first series of conflicts in what would be 682 years of Roman–Persian Wars'

In [242]:
textos["TextoLimpio"][0]

"['the', 'roman–parthian', 'war', '54bc', '–', '217ad', 'seri', 'conflict', 'parthian', 'empir', 'roman', 'republ', 'roman', 'empir', 'It', 'first', 'seri', 'conflict', 'would', '682', 'year', 'roman–persian', 'war']"

In [243]:
type(stop)

list

In [245]:
"the" in stop

True

In [268]:
textos.to_csv("Historia.csv")

In [269]:
textos = pd.read_csv("Historia.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",477.0,601.0,450.0,1131.0,89.0,NaN,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",980.0,926.0,571.0,1336.0,117.0,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"['113ad', 'roman', 'emperor', 'trajan', 'made'...",1079.0,733.0,508.0,787.0,68.0,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"['195', 'anoth', 'roman', 'invas', 'mesopotami...",448.0,435.0,161.0,277.0,31.0,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",483.0,594.0,523.0,933.0,196.0,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"['paleolith', 'peopl', 'suffer', 'less', 'fami...",322.0,476.0,436.0,728.0,1978.0,NaN,5
918,Large-seeded legumes were part of the human di...,"['large-seed', 'legum', 'part', 'human', 'diet...",786.0,830.0,842.0,1353.0,4099.0,NaN,5
919,In February 2019 scientists reported evidence ...,"['februari', '2019', 'scientist', 'report', 'e...",372.0,785.0,551.0,1174.0,3593.0,NaN,5
920,Anthropologists such as Tim White suggest that...,"['anthropologist', 'tim', 'white', 'suggest', ...",232.0,336.0,316.0,481.0,2008.0,NaN,5


In [271]:
columna = 2
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["Predicción"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

Comparando CLASE y features 921 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",477.0,601.0,450.0,1131.0,89.0,False,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",980.0,926.0,571.0,1336.0,117.0,False,1
2,In 113AD the Roman Emperor Trajan made eastern...,"['113ad', 'roman', 'emperor', 'trajan', 'made'...",1079.0,733.0,508.0,787.0,68.0,True,1
3,In 195 another Roman invasion of Mesopotamia b...,"['195', 'anoth', 'roman', 'invas', 'mesopotami...",448.0,435.0,161.0,277.0,31.0,True,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",483.0,594.0,523.0,933.0,196.0,False,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"['paleolith', 'peopl', 'suffer', 'less', 'fami...",322.0,476.0,436.0,728.0,1978.0,True,5
918,Large-seeded legumes were part of the human di...,"['large-seed', 'legum', 'part', 'human', 'diet...",786.0,830.0,842.0,1353.0,4099.0,True,5
919,In February 2019 scientists reported evidence ...,"['februari', '2019', 'scientist', 'report', 'e...",372.0,785.0,551.0,1174.0,3593.0,True,5
920,Anthropologists such as Tim White suggest that...,"['anthropologist', 'tim', 'white', 'suggest', ...",232.0,336.0,316.0,481.0,2008.0,True,5


In [272]:
textos.groupby(by=["Predicción"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,CLASE
Predicción,,,,,,,,
False,180,180,180,180,180,180,180,180
True,742,742,742,742,742,742,742,742


In [274]:
rate = 100 * 742 / (180 + 742)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 80.4772234273319


In [276]:
pifiadas = textos[textos["Predicción"] == False]
pifiadas

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",477.0,601.0,450.0,1131.0,89.0,False,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",980.0,926.0,571.0,1336.0,117.0,False,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",483.0,594.0,523.0,933.0,196.0,False,1
6,After 90BC the Parthian power was diminished b...,"['90bc', 'parthian', 'power', 'diminish', 'dyn...",339.0,276.0,248.0,427.0,65.0,False,1
7,When Pompey took charge of the war in the East...,"['pompey', 'took', 'charg', 'war', 'east', 're...",450.0,407.0,567.0,750.0,45.0,False,1
...,...,...,...,...,...,...,...,...,...
871,During the preceding Pliocene continents had c...,"['preced', 'pliocen', 'contin', 'continu', 'dr...",109.0,205.0,183.0,590.0,295.0,False,5
872,Most of Central America formed during the Plio...,"['central', 'america', 'form', 'pliocen', 'con...",159.0,290.0,128.0,594.0,301.0,False,5
873,Climates during the Pliocene became cooler and...,"['climat', 'pliocen', 'becam', 'cooler', 'drie...",325.0,529.0,562.0,1112.0,880.0,False,5
874,The effects of glaciation were global Antarcti...,"['effect', 'glaciat', 'global', 'antarctica', ...",367.0,596.0,369.0,1075.0,1013.0,False,5


In [278]:
pifiadas.groupby(by=["CLASE"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción
CLASE,,,,,,,,
1,41,41,41,41,41,41,41,41
2,79,79,79,79,79,79,79,79
3,47,47,47,47,47,47,47,47
5,13,13,13,13,13,13,13,13


In [284]:
cont = 0
cantTrues = 0
for i in pifiadas.index:
    if pifiadas["Predicción"][i]  ==  (np.argmax(pifiadas.iloc[cont,columna:columna + clases]) + 1):
       cantTrues +=1                               
    cont += 1
print("Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia:",cantTrues)
print("Cantidad de total de predicciones pifiadas:",cont)
print("Rate Freq/Pifios:", int(100 * cantTrues/cont)) 

Cantidad de veces que la predicción coincide con la clase con la máxima frecuencia: 0
Cantidad de total de predicciones pifiadas: 180
Rate Freq/Pifios: 0


In [287]:
# frecuenciasMaximas: diccionario que relaciona caa palabra con la clase
# que tiene la máxima frecuencia de aparición de esa palabra

frecuenciasMaximas = {} 

for word in vocabulario:    
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)   
        
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)    
    for k in range(1, len(freqWord)):
        frecuenciasMaximas[word] = np.argmax(freqWord)   


print()    
print("Frecuencias máximas")
for key in frecuenciasMaximas.keys():
    print(key,frecuenciasMaximas[key])    

clasesPalabrasFM = {}
for k in range(1,clases + 1):
    clasesPalabrasFM[k] = []     
for word in frecuenciasMaximas.keys():
    clasesPalabrasFM[frecuenciasMaximas[word]].append(word)
    
totV = 0    

# Verificamos que todas las palabras del vocabulario estén en el diciconario de 
# frecuencias de clases

for k in range(1, clases + 1):
    totV += len(clasesPalabrasFM[k])  
assert(totV == len(vocabulario)),"PALABRAS FALTANTES"    


Frecuencias máximas
avoid 3
view 3
km 5
destruct 3
przasnysz 4
aristotle 2
reintroduc 4
olañeta—gath 4
consent 3
ceasefir 4
castilian 3
1696 3
hussit 2
thracian 1
textil 4
compromise: 3
philip 3
rubinstein 4
sixth 2
legend 3
pensionari 3
island 5
invulner 4
told 4
51bc 1
concert 3
subsist 5
rip 1
pogrom 4
apocryph 3
lighter 2
inflat 3
mbuti 5
hispania 2
enamel 1
bergmann 4
bloodbath 2
exclus 1
minimis 3
multi-part 5
stratego 2
provid 4
giovanni 3
illustr 2
georg 4
linear 2
rendezv 3
weigh 4
gerbert 2
white 4
ludendorff 4
vinča 5
loos 2
ballestero 4
safeti 4
starv 4
class 4
crato; 3
shoulder 4
pisco 4
bargain 3
francia 2
underground 5
debt-leverag 3
geologist 4
task 3
čabrinović 4
veil 4
puerto 4
brussels 3
fritzlar 2
camp 4
troubl 3
conservative: 1
confidant 4
elit 4
châteauneuf 3
seizur 3
paleo-indian 5
departur 3
iron 1
unmarri 4
linguist 1
shipwreck 2
latin-speak 2
orang 3
demot 3
classic 2
massalia 1
imperialist 3
canning 4
man 1
clown 2
q-celtic 1
miguel 4
hugh 4
surgeon 3
vallad

equit 4
nitrogen 4
toll 3
valir 1
court 3
vladivostok 4
verdun 4
invas 1
philipstown 3
deathb 3
week 4
often 4
1576 3
advertis 4
warn 4
zero 2
freed 2
mascarenha 3
insid 4
light 4
artifact 5
sue 4
concept 2
boethiu 2
ruse 4
depict 3
supernatur 5
wrote 4
unsympathetic; 3
intrigu 3
fell 4
daraa 4
lugdunensi 1
longship 2
seleucid–parthian 1
rhetor 2
surround 1
farm 5
vertic 1
clausum 3
gangren 3
either 3
1559–1568 3
escal 3
brest-litovsk 4
median 1
1547 2
librari 2
sordid 3
own 4
fistula 3
vatican 2
1999 5
exhaust 4
70 5
1916; 4
usual 4
intensifi 3
earnest 3
salic 2
francs-tireur 4
say 1
crimin 3
1897 1
enver 4
1866 4
discount 4
south-west 4
fixed-w 4
isthmu 5
haughti 3
temperatur 5
infanta 3
sindh 2
day; 4
climat 5
continent 1
swiss 2
manuel 3
coordin 4
protect 4
ariu 2
filip 3
dissip 4
statu 3
dress 1
smash 3
predat 5
cathedr 2
hypothesi 5
captured; 1
1887 4
qingdao 4
entitl 3
mitig 5
'gloire 3
25em;height:1 2
reservoir 4
physiocrat 4
ali 2
impedi 2
maglemosian 5
frisia 3
iberian 1
fore

de-urban 2
typhu 4
washington 4
ascend 2
alani 2
penetr 2
physiqu 1
genu 5
florenc 2
size 5
caus 4
kiel 4
costli 4
correctli 4
masqu 2
cautiou 3
787 2
threw 2
wage 3
massilia 1
630 2
end 4
derbent 2
counterpart 1
atroc 4
smaller 4
43 3
childbirth 3
600 4
tower 2
holland 3
stereotyp 1
cone 4
refuge 4
space 5
eight 4
easi 4
necessari 4
rapid 2
pipe 5
friedrich 4
forward-throw 4
anatolia 1
likewis 5
gibbon 2
wrest 3
parthian 1
associ 1
catarina 3
ideal 2
slovak 1
meat 5
vii 4
duti 4
1917–18 4
primit 5
februari 4
neutrality; 4
fifteenth 2
stabilis 4
adulation; 3
1918–1919 4
example: 2
pagan 2
primat 5
πόλις 1
captur 4
carolingian 2
tiberia 4
agenc 4
603a 1
royal 3
1871 4
suggest 5
illiter 2
demograph 2
crosswis 1
demolish 3
900–9 5
entertain 3
discuss 3
blenheim 3
prehistor 5
food 5
manila 4
clemen 3
alvarez 3
moula 5
14th 2
domini 2
ice-raft 5
excurs 5
diego 4
notat 2
axe 5
pillag 2
sweden 3
100-man 4
supplant 4
infantri 4
seriou 4
creator 4
pronunci 1
mid-11th 2
setback 2
sicilian 2
fina

apotropa 5
limewat 1
patern 1
mariner 2
12th-centuri 1
churchil 3
inherit 2
baden 3
750 2
attrit 4
wast 4
embark 2
da 2
peer 2
compass 2
mobilisation; 4
profession 2
hôtel 3
site 5
vittorio 4
cologn 3
lifeboat—carri 4
frey 1
378 2
militar 4
deficit 3
1497 2
recreat 4
subjug 2
archiv 4
pamplona 3
dim 4
eurasia 5
juan 3
upend 4
ironwork 3
gallu 1
saviour 4
sister-in-law 3
allianc 4
romanis 1
tuscan 2
novi 3
cultur 1
cavalri 2
tie 3
túpac 4
viceroyalti 4
nürnberg 4
miracl 3
oper 4
mary 3
appointe 4
parthian-roman 1
iran 4
spearhead 4
went 4
rye 2
spanish-american 4
26–30 4
25 4
christ 2
gene 1
bewitch 4
graciou 3
1455–1485 2
naarden 3
munit 4
benefic 4
bushfood 5
time 3
tlingit 5
yell 4
reign 3
1559 3
recognis 3
llandoveri 4
1832 4
multidisciplinari 1
tuni 3
1920 4
effici 5
manoeuvr 3
sylvest 2
two-front 4
2–2 4
circumv 2
turin 3
1811 4
lyon 3
vehement 3
clumsi 4
8% 4
brandenburg-prussia 3
titl 3
apostol 3
tacitu 1
obes 1
stretch 4
1899 4
repeat 1
95 2
tast 1
bernini 3
sviatoslav 2
superb

rampant 3
report 4
bulgarian: 1
boil 3
onto 2
carrha 1
1588 3
timefram 5
drift 5
unfold 4
brain 5
acced 2
riga 4
ruiz 1
£5 4
erupt 3
retain 3
think 1
hinder 4
histori 2
forgotten 4
johan 1
muck 2
abbāsid 2
legend{page-break-inside:avoid;break-inside:avoid-column} 2
ages: 2
maria 3
main 4
$58 4
scharnhorst 4
córdoba 4
dazzl 3
cooley 1
sall 3
leo 2
forth 3
late 2
anthropolog 5
bureau 4
8th 2
leather 5
lower/middl 5
carbohydr 5
mention 1
presidi 3
placat 3
transfer 3
graveyard 4
southwestern 1
1597 3
majorca 3
atlantic—th 4
servic 4
detail 2
propon 4
captainci 4
max 4
giusti 4
relent 3
bogotá 4
wear 1
arsac 1
longer 4
concordia 3
turcoman 4
admiss 4
smoothli 4
trip 4
prussian 4
1315 2
matrilin 5
860 4
1667 3
plantinga 3
alexandrian 2
offici 3
lepanto 3
diputación 3
outright 4
hatra 1
formid 2
teatro 4
interspers 2
quarrel 2
1ad 1
upheav 2
full-scal 4
kressenstein 4
pre-liter 5
rhythmic 5
124bc 1
wycliff 2
azerbaijani 4
almost 4
nôtre 3
itiner 2
depth 4
regnal 2
152 4
sunday 4
1658 3
domin

collect 3
reconquest 4
kingdoms: 2
carpathian 4
geneticist 5
abstract 2
tidal 1
earli 5
deepest 4
sive 3
1881 4
arrog 3
shell-torn 4
ukrain 4
frugivor 5
cleph 2
ideolog 4
hairi 1
nivel 4
740-748 2
compet 2
probabl 5
newspap 4
970-971 2
sea; 3
out-of-d 4
save 4
brittani 1
iliad 2
840 5
distantli 3
derrynaflan 1
well-known 3
león 1
internazionalista 4
clay 5
primari 4
blond 1
gain 4
distinct 1
disputed; 1
prior 5
times; 3
confid 3
piero 3
member 3
suspicion 3
124–123 1
thick 4
30% 4
transubstanti 2
theodosiu 2
friction 4
sexual 5
piar 4
eyewit 3
stabl 2
pattern 1
rich 1
broach 3
postul 1
throw 1
heathen 2
patriot 4
magellan 3
hostag 4
magazin 4
societi 5
spectacl 1
1914–1918 4
1376 2
belliger 4
fundament 5
worm 2
fernández 4
solitari 4
flank 4
famili 3
champion 2
kingdoms; 3
knock 4
trackway 1
triumphs: 4
sub-saharan 5
alexand 3
this; 3
thu 3
ethiopia 5
frozen 5
dispar 3
char 5
ordin 3
lausann 4
osro 1
specul 2
canzonier 2
69bc 1
voic 4
equival 4
netherlands; 3
reorgan 3
roman 1
61 5
ski

austrian 4
groups: 1
two-or-thre 4
liberators 1
217ad 1
letter-writ 4
reinstat 3
douan 3
caliph 2
place; 4
illyria 2
33 1
corbulo 1
thing 2
tip 5
south-east 1
reach 5
re-arm 4
voltaire 3
behind 2
sixty-year 3
eleventh 4
regain 4
imageri 5
iii; 1
ziyad 2
front; 4
successful: 4
khazars—to 2
bestow 1
corsica 3
biped 5
churchman 3
period 2
people 3
exhibit 1
steven 5
prolong 2
influenti 1
deer 5
hand 3
unleash 4
ambilin 5
lack 4
strong 4
musician 3
ġgantija 5
monster 3
cornish 1
raft 5
banat 4
pro-french 3
517bc 1
were: 3
1–4 4
sopwith 4
umayyad 2
170 4
wind 4
sieur 3
non-intervent 4
vastli 2
zúñiga 3
larger 4
66 1
empire 4
protectionist 4
1669 3
unlik 3
anger 3
non-bind 4
ochr 5
silent 3
gaiu 1
result 4
bring 4
medina 4
assert 1
1921 4
milan 3
1647 3
quintu 1
advers 3
philipp 3
granduncl 3
fontain 3
genr 2
galicia 4
shandong 4
brutu 1
stori 1
swath 2
enrag 3
mid-1919 4
neighbor 4
zborov 4
scar 4
bracelet 5
infiltr 4
peru 4
gorbea 1
cambrai—th 4
legum 5
perhap 5
enough 3
aerial 4
limelight

In [297]:
frecuenciasMaxDetalle = {}     
    
for word, value in frecuenciasMaximas.items():
    freqWord = []
    for k in range(clases + 1):
        freqWord.append(0)     
    for k in range(1, len(freqWord)):
        freqWord[k] = frecuencias.get((word, k),0)        
    
    if sum(freqWord) == frecuencias[word, np.argmax(freqWord)]:
        rate = 99
    else:    
        rate = int(100 * frecuencias[word, np.argmax(freqWord)] / sum(freqWord))
    
    frecuenciasMaxDetalle[word] = value, rate - 19
    
print()
print("frecuenciasMaxDetalle")
print()
cont = 0
nconfusas = 0
confusas = []
for key in frecuenciasMaxDetalle.keys():
    cont +=1
    if cont > 100000:
        break
    if frecuenciasMaxDetalle[key][1] < 60:    
        print("PALABRA CONFUSA: ",key, frecuenciasMaxDetalle[key])   
        print(frecuenciasMaximas[key])
        freqWord = []
        for k in range(clases + 1):
            freqWord.append(0)   
        for k in range(1, len(freqWord)):
            freqWord[k] = frecuencias.get((key, k),0)
        
        # print("Mínimo",np.min(freqWord[1:]))
        # print("Suma",sum(freqWord[1:]))
        print()
        rateMinSum =  int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]))
        if rateMinSum > 25:
            nconfusas += 1
            print(freqWord[1:])
            print("minimo",np.min(freqWord[1:])  )
            print(sum(freqWord[1:]))
            print(freqWord[frecuenciasMaxDetalle[key][0]])
            print("% Mínimo / suma:", int(500 *  np.min(freqWord[1:]) / sum(freqWord[1:]) ) )
            # print("key",type(key),key)
            # print("key.shape",key.split().shape)
            confusas.append(key)
        
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)
confusas
print("cantidad de palabras confusas = ",nconfusas)   
len(confusas)


frecuenciasMaxDetalle

PALABRA CONFUSA:  avoid (3, 21)
3

[3, 2, 10, 8, 2]
minimo 2
25
10
% Mínimo / suma: 40
PALABRA CONFUSA:  view (3, 7)
3

[7, 5, 8, 6, 4]
minimo 4
30
8
% Mínimo / suma: 66
PALABRA CONFUSA:  km (5, 38)
5

PALABRA CONFUSA:  destruct (3, 56)
3

PALABRA CONFUSA:  consent (3, 41)
3

PALABRA CONFUSA:  legend (3, 43)
3

PALABRA CONFUSA:  island (5, 8)
5

[8, 4, 8, 6, 10]
minimo 4
36
10
% Mínimo / suma: 55
PALABRA CONFUSA:  pogrom (4, 47)
4

PALABRA CONFUSA:  hispania (2, 56)
2

PALABRA CONFUSA:  exclus (1, 31)
1

PALABRA CONFUSA:  minimis (3, 31)
3

PALABRA CONFUSA:  provid (4, 16)
4

[4, 5, 7, 14, 9]
minimo 4
39
14
% Mínimo / suma: 51
PALABRA CONFUSA:  giovanni (3, 47)
3

PALABRA CONFUSA:  illustr (2, 31)
2

PALABRA CONFUSA:  white (4, 47)
4

PALABRA CONFUSA:  loos (2, 31)
2

PALABRA CONFUSA:  safeti (4, 47)
4

PALABRA CONFUSA:  class (4, 36)
4

PALABRA CONFUSA:  task (3, 14)
3

PALABRA CONFUSA:  camp (4, 56)
4

PALABRA CONFUSA:  elit (4, 39)
4

[1, 2, 3, 10, 1]
minimo 

% Mínimo / suma: 31
PALABRA CONFUSA:  350 (3, 31)
3

PALABRA CONFUSA:  erad (2, 31)
2

PALABRA CONFUSA:  resurrect (2, 31)
2

PALABRA CONFUSA:  along (4, 31)
4

PALABRA CONFUSA:  laid (3, 43)
3

PALABRA CONFUSA:  work (3, 13)
3

[8, 12, 17, 8, 8]
minimo 8
53
17
% Mínimo / suma: 75
PALABRA CONFUSA:  met (4, 21)
4

PALABRA CONFUSA:  distribut (3, 31)
3

PALABRA CONFUSA:  physic (4, 18)
4

PALABRA CONFUSA:  rel (5, 22)
5

[5, 5, 2, 6, 13]
minimo 2
31
13
% Mínimo / suma: 32
PALABRA CONFUSA:  defin (2, 11)
2

PALABRA CONFUSA:  actual (3, 17)
3

PALABRA CONFUSA:  immedi (4, 50)
4

PALABRA CONFUSA:  modest (3, 31)
3

PALABRA CONFUSA:  offic (4, 46)
4

PALABRA CONFUSA:  strain (3, 31)
3

PALABRA CONFUSA:  field (2, 26)
2

PALABRA CONFUSA:  oil (4, 41)
4

PALABRA CONFUSA:  notabl (3, 4)
3

[2, 2, 3, 3, 3]
minimo 2
13
3
% Mínimo / suma: 76
PALABRA CONFUSA:  song (1, 14)
1

PALABRA CONFUSA:  potenti (4, 56)
4

PALABRA CONFUSA:  consult (2, 14)
2

PALABRA CONFUSA:  austria (3, 26)
3

PALABRA CONFU

PALABRA CONFUSA:  famous (1, 31)
1

PALABRA CONFUSA:  upper (5, 59)
5

PALABRA CONFUSA:  studi (2, 37)
2

PALABRA CONFUSA:  secur (4, 31)
4

PALABRA CONFUSA:  effect (4, 32)
4

PALABRA CONFUSA:  franc (3, 40)
3

PALABRA CONFUSA:  spanish: (3, 47)
3

PALABRA CONFUSA:  select (4, 47)
4

PALABRA CONFUSA:  switzerland (1, 56)
1

PALABRA CONFUSA:  crucial (3, 21)
3

PALABRA CONFUSA:  sympathet (5, 56)
5

PALABRA CONFUSA:  common (1, 11)
1

PALABRA CONFUSA:  potteri (1, 14)
1

PALABRA CONFUSA:  importantli (1, 31)
1

PALABRA CONFUSA:  obtain (5, 31)
5

PALABRA CONFUSA:  respect (4, 28)
4

PALABRA CONFUSA:  abdic (4, 58)
4

PALABRA CONFUSA:  fortun (2, 31)
2

PALABRA CONFUSA:  qualiti (5, 41)
5

PALABRA CONFUSA:  coron (3, 47)
3

PALABRA CONFUSA:  cold (5, 21)
5

PALABRA CONFUSA:  past (4, 51)
4

PALABRA CONFUSA:  counsel (3, 31)
3

PALABRA CONFUSA:  justic (3, 56)
3

PALABRA CONFUSA:  overcom (3, 31)
3

PALABRA CONFUSA:  oust (1, 14)
1

PALABRA CONFUSA:  forcibl (3, 31)
3

PALABRA CONFUSA:  


PALABRA CONFUSA:  chronolog (1, 31)
1

PALABRA CONFUSA:  virtual (3, 25)
3

PALABRA CONFUSA:  back (4, 24)
4

PALABRA CONFUSA:  reward (3, 47)
3

PALABRA CONFUSA:  motiv (2, 14)
2

PALABRA CONFUSA:  alien (3, 47)
3

PALABRA CONFUSA:  wild (5, 54)
5

PALABRA CONFUSA:  templ (1, 31)
1

PALABRA CONFUSA:  sens (3, 19)
3

PALABRA CONFUSA:  dioces (2, 31)
2

PALABRA CONFUSA:  made (4, 20)
4

[12, 13, 21, 36, 10]
minimo 10
92
36
% Mínimo / suma: 54
PALABRA CONFUSA:  one (3, 14)
3

[16, 18, 34, 29, 6]
minimo 6
103
34
% Mínimo / suma: 29
PALABRA CONFUSA:  certain (2, 12)
2

[2, 5, 5, 2, 2]
minimo 2
16
5
% Mínimo / suma: 62
PALABRA CONFUSA:  aggressor (1, 31)
1

PALABRA CONFUSA:  hunter (5, 47)
5

PALABRA CONFUSA:  decre (3, 31)
3

PALABRA CONFUSA:  swiftli (1, 31)
1

PALABRA CONFUSA:  retook (2, 14)
2

PALABRA CONFUSA:  nation (4, 55)
4

PALABRA CONFUSA:  viii (2, 31)
2

PALABRA CONFUSA:  mission (3, 22)
3

PALABRA CONFUSA:  subject (3, 18)
3

[4, 3, 9, 5, 3]
minimo 3
24
9
% Mínimo / suma: 62


PALABRA CONFUSA:  embark (2, 31)
2

PALABRA CONFUSA:  profession (2, 31)
2

PALABRA CONFUSA:  378 (2, 31)
2

PALABRA CONFUSA:  juan (3, 56)
3

PALABRA CONFUSA:  allianc (4, 26)
4

PALABRA CONFUSA:  cultur (1, 33)
1

PALABRA CONFUSA:  cavalri (2, 31)
2

PALABRA CONFUSA:  tie (3, 21)
3

PALABRA CONFUSA:  went (4, 27)
4

[1, 2, 4, 7, 1]
minimo 1
15
7
% Mínimo / suma: 33
PALABRA CONFUSA:  25 (4, 31)
4

PALABRA CONFUSA:  time (3, 9)
3

[13, 33, 39, 30, 22]
minimo 13
137
39
% Mínimo / suma: 47
PALABRA CONFUSA:  reign (3, 55)
3

PALABRA CONFUSA:  recognis (3, 44)
3

PALABRA CONFUSA:  tuni (3, 47)
3

PALABRA CONFUSA:  effici (5, 38)
5

PALABRA CONFUSA:  manoeuvr (3, 31)
3

PALABRA CONFUSA:  titl (3, 27)
3

PALABRA CONFUSA:  stretch (4, 31)
4

PALABRA CONFUSA:  repeat (1, 14)
1

PALABRA CONFUSA:  tast (1, 31)
1

PALABRA CONFUSA:  erect (3, 31)
3

PALABRA CONFUSA:  ferdinand (4, 51)
4

PALABRA CONFUSA:  blood (3, 56)
3

PALABRA CONFUSA:  henc (3, 31)
3

PALABRA CONFUSA:  especi (2, 8)
2

PALABRA

...

PALABRA CONFUSA:  first (4, 15)
4

[15, 34, 26, 51, 21]
minimo 15
147
51
% Mínimo / suma: 51
PALABRA CONFUSA:  symbol (5, 25)
5

PALABRA CONFUSA:  behead (1, 47)
1

PALABRA CONFUSA:  grant (3, 23)
3

PALABRA CONFUSA:  undertook (1, 14)
1

PALABRA CONFUSA:  sometim (5, 12)
5

[4, 8, 5, 3, 9]
minimo 3
29
9
% Mínimo / suma: 51
PALABRA CONFUSA:  joachim (3, 31)
3

PALABRA CONFUSA:  extrem (2, 11)
2

PALABRA CONFUSA:  highli (3, 31)
3

PALABRA CONFUSA:  rivalri (4, 31)
4

PALABRA CONFUSA:  collabor (4, 47)
4

PALABRA CONFUSA:  concess (3, 31)
3

PALABRA CONFUSA:  mint (1, 56)
1

PALABRA CONFUSA:  courag (1, 31)
1

PALABRA CONFUSA:  absenc (4, 31)
4

PALABRA CONFUSA:  fit (2, 31)
2

PALABRA CONFUSA:  economi (4, 39)
4

PALABRA CONFUSA:  tent (2, 31)
2

PALABRA CONFUSA:  siberia (5, 47)
5

PALABRA CONFUSA:  sardinia (3, 56)
3

PALABRA CONFUSA:  imposs (1, 14)
1

PALABRA CONFUSA:  jealous (2, 31)
2

PALABRA CONFUSA:  versail (3, 41)
3

PALABRA CONFUSA:  last (4, 15)
4

[4, 14, 18, 21, 4]
minim

minimo 4
32
9
% Mínimo / suma: 62
PALABRA CONFUSA:  hundr (2, 28)
2

PALABRA CONFUSA:  agre (4, 24)
4

PALABRA CONFUSA:  answer (2, 14)
2

PALABRA CONFUSA:  die (3, 33)
3

PALABRA CONFUSA:  abl (4, 16)
4

[3, 6, 7, 10, 2]
minimo 2
28
10
% Mínimo / suma: 35
PALABRA CONFUSA:  time; (3, 31)
3

PALABRA CONFUSA:  chosen (4, 31)
4

PALABRA CONFUSA:  quentin (3, 56)
3

PALABRA CONFUSA:  urban (2, 27)
2

PALABRA CONFUSA:  democrat (4, 31)
4

PALABRA CONFUSA:  impress (4, 41)
4

PALABRA CONFUSA:  separ (4, 19)
4

[4, 3, 5, 10, 4]
minimo 3
26
10
% Mínimo / suma: 57
PALABRA CONFUSA:  ecclesiast (2, 41)
2

PALABRA CONFUSA:  pictur (1, 31)
1

PALABRA CONFUSA:  local (4, 25)
4

PALABRA CONFUSA:  unknown (5, 25)
5

PALABRA CONFUSA:  16 (4, 31)
4

PALABRA CONFUSA:  1545 (2, 31)
2

PALABRA CONFUSA:  albert (4, 56)
4

PALABRA CONFUSA:  reconstruct (1, 31)
1

PALABRA CONFUSA:  deliber (4, 47)
4

PALABRA CONFUSA:  strasbourg (3, 56)
3

PALABRA CONFUSA:  defend (4, 31)
4

PALABRA CONFUSA:  clear (4, 41)
4


PALABRA CONFUSA:  commonwealth (2, 31)
2

PALABRA CONFUSA:  approv (4, 47)
4

PALABRA CONFUSA:  advis (3, 47)
3

PALABRA CONFUSA:  macedonian (2, 56)
2

PALABRA CONFUSA:  injuri (3, 31)
3

PALABRA CONFUSA:  50 (4, 12)
4

[1, 3, 2, 5, 5]
minimo 1
16
5
% Mínimo / suma: 31
PALABRA CONFUSA:  left (4, 23)
4

PALABRA CONFUSA:  waterway (2, 31)
2

PALABRA CONFUSA:  like (4, 7)
4

[11, 9, 13, 17, 14]
minimo 9
64
17
% Mínimo / suma: 70
PALABRA CONFUSA:  son-in-law (1, 31)
1

PALABRA CONFUSA:  discoveri (2, 31)
2

PALABRA CONFUSA:  yet (2, 28)
2

PALABRA CONFUSA:  began (4, 22)
4

[10, 21, 16, 43, 14]
minimo 10
104
43
% Mínimo / suma: 48
PALABRA CONFUSA:  crisi (4, 35)
4

PALABRA CONFUSA:  westward (1, 47)
1

PALABRA CONFUSA:  inquisit (3, 56)
3

PALABRA CONFUSA:  fair (2, 47)
2

PALABRA CONFUSA:  synod (1, 31)
1

PALABRA CONFUSA:  equip (2, 31)
2

PALABRA CONFUSA:  princip (4, 38)
4

PALABRA CONFUSA:  settl (2, 7)
2

[4, 5, 2, 4, 4]
minimo 2
19
5
% Mínimo / suma: 52
PALABRA CONFUSA:  drive (4, 

PALABRA CONFUSA:  joseph (3, 47)
3

PALABRA CONFUSA:  non-liter (2, 31)
2

PALABRA CONFUSA:  divert (4, 47)
4

PALABRA CONFUSA:  experienc (2, 18)
2

PALABRA CONFUSA:  flat (1, 31)
1

cantidad de palabras confusas =  274
cantidad de palabras confusas =  274


274

In [298]:
len(vocabulario)

9070

In [302]:
len(confusas)

274

In [301]:
textos["TextoLimpio"][0]

"['roman–parthian', 'war', '54bc', '–', '217ad', 'seri', 'conflict', 'parthian', 'empir', 'roman', 'republ', 'roman', 'empir', 'first', 'seri', 'conflict', 'would', '682', 'year', 'roman–persian', 'war']"

## Para extraer las palabras confusas debemos hacerlo sobre el dataset total, con y sin pifiadas  

In [311]:
textos = pd.read_csv("MiniHistoriaTextosProcesados.csv")
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"['roman–parthian', 'war', '54bc', '–', '217ad'...",NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,"['battl', 'parthian', 'empir', 'roman', 'repub...",NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"['113ad', 'roman', 'emperor', 'trajan', 'made'...",NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"['195', 'anoth', 'roman', 'invas', 'mesopotami...",NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"['triumph', 'seleucid–parthian', 'war', 'annex...",NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"['paleolith', 'peopl', 'suffer', 'less', 'fami...",NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,"['large-seed', 'legum', 'part', 'human', 'diet...",NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,"['februari', '2019', 'scientist', 'report', 'e...",NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,"['anthropologist', 'tim', 'white', 'suggest', ...",NaN,NaN,NaN,NaN,NaN,NaN,5


In [312]:
for i in textos.index:
    lista = []
    for word in textos["TextoLimpio"][i][:-1].split(","):
        word = word[2:-1]
        while word[-1:] == "'" or word[-1:] == '"':
            word = limpiarWord(word)
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"    
        if word not in confusas:
            lista.append(word) 
    clear_output( wait = True )
    print(f"Procesando {i} de {len(textos.index)}") 
    print()
    print("largo texto:",len(textos["TextoLimpio"][i]))
    print("largo lista sin confusas:",len(lista))
    print()
    textos["TextoLimpio"][i] = lista
textos

Procesando 921 de 922

largo texto: 191
largo lista sin confusas: 13



,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"[roman–parthian, war, 54bc, 217ad, seri, confl...",NaN,NaN,NaN,NaN,NaN,NaN,1
1,Battles between the Parthian Empire and the Ro...,"[battl, parthian, empir, roman, republ, 54bc, ...",NaN,NaN,NaN,NaN,NaN,NaN,1
2,In 113AD the Roman Emperor Trajan made eastern...,"[113ad, roman, emperor, trajan, eastern, conqu...",NaN,NaN,NaN,NaN,NaN,NaN,1
3,In 195 another Roman invasion of Mesopotamia b...,"[195, anoth, roman, invas, mesopotamia, empero...",NaN,NaN,NaN,NaN,NaN,NaN,1
4,After triumphing in the Seleucid–Parthian wars...,"[triumph, seleucid–parthian, war, annex, amoun...",NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"[paleolith, suffer, famin, malnutrit, neolith,...",NaN,NaN,NaN,NaN,NaN,NaN,5
918,Large-seeded legumes were part of the human di...,"[large-seed, legum, human, diet, neolith, revo...",NaN,NaN,NaN,NaN,NaN,NaN,5
919,In February 2019 scientists reported evidence ...,"[februari, 2019, scientist, report, evid, isot...",NaN,NaN,NaN,NaN,NaN,NaN,5
920,Anthropologists such as Tim White suggest that...,"[anthropologist, tim, white, suggest, cannib, ...",NaN,NaN,NaN,NaN,NaN,NaN,5


## Ahora que el texto ha quedado sin las palabras confusas vamos a volver a extraer los features  

In [315]:
for i in textos.index:
    print(i)
    palabras = []    
    for word in textos["TextoLimpio"][i]:
        assert (word in vocabulario),"La palabra " + word + " no está en el vocabulario!"     
        palabras.append(word)    
        for k in range(1, clases + 1):
            textos["Clase" + str(k)][i] = etiquetas(palabras,frecuencias,clases)[0,k]
    clear_output( wait = True )
    print(f"Creando Features {i} de {len(textos.index)}") 
textos

Creando Features 921 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"[roman–parthian, war, 54bc, 217ad, seri, confl...",447.0,529.0,375.0,1028.0,25.0,True,1
1,Battles between the Parthian Empire and the Ro...,"[battl, parthian, empir, roman, republ, 54bc, ...",910.0,831.0,468.0,1163.0,31.0,True,1
2,In 113AD the Roman Emperor Trajan made eastern...,"[113ad, roman, emperor, trajan, eastern, conqu...",1036.0,701.0,449.0,685.0,31.0,True,1
3,In 195 another Roman invasion of Mesopotamia b...,"[195, anoth, roman, invas, mesopotamia, empero...",436.0,412.0,140.0,218.0,13.0,True,1
4,After triumphing in the Seleucid–Parthian wars...,"[triumph, seleucid–parthian, war, annex, amoun...",371.0,414.0,360.0,699.0,76.0,True,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"[paleolith, suffer, famin, malnutrit, neolith,...",142.0,233.0,176.0,320.0,1749.0,False,5
918,Large-seeded legumes were part of the human di...,"[large-seed, legum, human, diet, neolith, revo...",370.0,263.0,349.0,690.0,3499.0,False,5
919,In February 2019 scientists reported evidence ...,"[februari, 2019, scientist, report, evid, isot...",143.0,442.0,224.0,592.0,3204.0,False,5
920,Anthropologists such as Tim White suggest that...,"[anthropologist, tim, white, suggest, cannib, ...",141.0,210.0,200.0,297.0,1859.0,False,5


In [316]:
columna = 2
for i in textos.index:
    print(textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1)
    textos["Predicción"][i] = textos.loc[i,"CLASE"] ==  np.argmax(textos.iloc[i,columna:columna + clases].values) + 1
    clear_output( wait = True )
    print(f"Comparando CLASE y features {i} de {len(textos.index)}") 
textos      

Comparando CLASE y features 921 de 922


,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,The Roman–Parthian Wars 54BC – 217AD were a se...,"[roman–parthian, war, 54bc, 217ad, seri, confl...",447.0,529.0,375.0,1028.0,25.0,False,1
1,Battles between the Parthian Empire and the Ro...,"[battl, parthian, empir, roman, republ, 54bc, ...",910.0,831.0,468.0,1163.0,31.0,False,1
2,In 113AD the Roman Emperor Trajan made eastern...,"[113ad, roman, emperor, trajan, eastern, conqu...",1036.0,701.0,449.0,685.0,31.0,True,1
3,In 195 another Roman invasion of Mesopotamia b...,"[195, anoth, roman, invas, mesopotamia, empero...",436.0,412.0,140.0,218.0,13.0,True,1
4,After triumphing in the Seleucid–Parthian wars...,"[triumph, seleucid–parthian, war, annex, amoun...",371.0,414.0,360.0,699.0,76.0,False,1
...,...,...,...,...,...,...,...,...,...
917,Paleolithic peoples suffered less famine and m...,"[paleolith, suffer, famin, malnutrit, neolith,...",142.0,233.0,176.0,320.0,1749.0,True,5
918,Large-seeded legumes were part of the human di...,"[large-seed, legum, human, diet, neolith, revo...",370.0,263.0,349.0,690.0,3499.0,True,5
919,In February 2019 scientists reported evidence ...,"[februari, 2019, scientist, report, evid, isot...",143.0,442.0,224.0,592.0,3204.0,True,5
920,Anthropologists such as Tim White suggest that...,"[anthropologist, tim, white, suggest, cannib, ...",141.0,210.0,200.0,297.0,1859.0,True,5


In [317]:
textos.groupby(by=["Predicción"]).count()

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,CLASE
Predicción,,,,,,,,
False,163,163,163,163,163,163,163,163
True,759,759,759,759,759,759,759,759


In [318]:
rate = 100 * 759 / (163 + 749)
print("Porcentaje de features acertados",rate)

Porcentaje de features acertados 83.22368421052632


**Este resultado es prometedor. Vamos a repetir esto mismos con el dataset completo**